In [ ]:
import requests
import pandas as pd


In [ ]:

# Base URL for the tournament's hole stats (replace with your specific tournament ID and round ID)
base_url = "https://www.pdga.com/api/v1/feat/stats/hole-stats-breakdown/122324129/"

# Number of holes in the course
num_holes = 18

# List to store data for all holes
all_holes_stats = []

for hole_num in range(1, num_holes + 1):
    # Construct the full URL for the current hole
    url = f"{base_url}{hole_num}"
    
    # Fetch the data
    response = requests.get(url)
    data = response.json()
    
    # Extract and store stats for each player in the current hole
    for entry in data:
        player_info = entry['liveScore']['liveResult']
        breakdown = entry['breakdown']
        
        # Combine player info with breakdown and add hole number
        player_stats = {
            'Hole': hole_num,
            'FirstName': player_info['firstName'],
            'LastName': player_info['lastName'],
            'PDGANum': player_info['pdgaNum'],
            'Place': player_info['place'],
            'TotalScore': player_info['total'],
            'ToPar': player_info['toPar'],
            'Driving': breakdown.get('driving', None),
            'Scramble': breakdown.get('scramble', None),
            'Green': breakdown.get('green', None),
            'C1x': breakdown.get('c1x', 0),
            'C1': breakdown.get('c1', 0),
            'C2': breakdown.get('c2', 0),
            'ThrowIn': breakdown.get('throwIn', 0),
            'OB': breakdown.get('ob', 0),
            'Hazard': breakdown.get('hazard', 0),
            'MissedMando': breakdown.get('missedMando', 0),
            'LostDisc': breakdown.get('lostDisc', 0),
            'Penalty': breakdown.get('penalty', 0),
            'HoleScore': breakdown.get('holeScore', None)
        }
        all_holes_stats.append(player_stats)

In [ ]:

# Convert the list to a DataFrame
df = pd.DataFrame(all_holes_stats)

# Display the data
print(df)


In [ ]:

# Save to CSV for further analysis
df.to_csv('tournament_hole_stats.csv', index=False)


In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
# URL of the PDGA page listing the tournaments (tournament search page sorted for ES and M tournaments within the date range of this year)
tournaments_url = 'https://www.pdga.com/tour/search?OfficialName=&td=&date_filter%5Bmin%5D%5Bdate%5D=2024-02-01&date_filter%5Bmax%5D%5Bdate%5D=2025-08-14&Tier%5B%5D=ES&Tier%5B%5D=M' 


In [ ]:

# Send a GET request to the page
response = requests.get(tournaments_url)
soup = BeautifulSoup(response.text, 'html.parser')


In [ ]:


# Initialize a dictionary to store tournament names and IDs
tournaments_dict = {}

# Find the table containing the events
table = soup.find('div', class_='table-container')


In [ ]:

# If the table exists, proceed
if table:
    # Find all rows in the table body
    rows = table.find('tbody').find_all('tr')

    for row in rows:
        # Find the cell with the tournament name
        tournament_cell = row.find('td', class_='views-field-OfficialName')
        
        if tournament_cell:
            # Extract the link and name
            link = tournament_cell.find('a')
            if link:
                tournament_name = link.text.strip()
                href = link['href']
                tournament_id = href.split('/')[-1]
                
                # Add to dictionary
                tournaments_dict[tournament_name] = tournament_id


In [ ]:

# Print the resulting dictionary
print(tournaments_dict)
# use this dictionary in asyncio_run.py to find round ids

In [ ]:
# round_ids captured by asyncio_run.py 
round_ids = {'78584': {1: '122299849'}, '77775': {1: '122287100', 2: '122287102', 3: '122287104'}, '77758': {1: '122301092', 2: '122301094', 3: '122301096', 4: '122314192'}, '77759': {1: '122288247', 2: '122288249', 3: '122288251'}, '77091': {}, '77760': {1: '122291884', 2: '122291886', 3: '122291888'}, '77761': {1: '122286664', 2: '122286666', 3: '122286668'}, '77762': {1: '122286804', 2: '122286806', 3: '122286808'}, '77099': {1: '122283440', 2: '122283442', 3: '122283444', 4: '122360073'}, '77763': {1: '122287578', 2: '122287580', 3: '122287582'}, '78193': {1: '122292625', 2: '122292627', 3: '122292629'}, '77764': {1: '122286793', 2: '122286794', 3: '122286795'}, '77765': {1: '122299697', 2: '122299699', 3: '122299701', 4: '122299703'}, '77766': {1: '122316840', 2: '122316842', 3: '122316844'}, '77093': {}, '77133': {}, '77094': {}, '78194': {1: '122292631', 2: '122292633', 3: '122292635'}, '78271': {1: '122312714', 2: '122312716', 3: '122312718'}, '78195': {1: '122292637', 2: '122292639', 3: '122292641'}, '77768': {1: '122301365', 2: '122301367', 3: '122301369'}, '78196': {1: '122292643', 2: '122292645', 3: '122292647'}, '77095': {}, '77750': {1: '122309308', 2: '122309309', 3: '122309310', 4: '122309312'}, '78197': {1: '122292649', 2: '122292651', 3: '122292653'}}

In [ ]:
tournament_rounds = {}
for tournament_name, tournament_id in tournaments_dict.items():
    for tid, rounds in round_ids.items():
        if tournament_id == tid:
            tournament_rounds[tournament_name] = round_ids[tournament_id]

print(tournament_rounds)

In [ ]:
cleaned_tournament_rounds = {}
for name, rids in tournament_rounds.items():
    if rids != {}:
        cleaned_tournament_rounds[name] = rids
        print(f"{name} round ID: {rids}")



In [ ]:
# Function to pull stats from each round with an associated round id
# Not ready yet 
def get_tournament_data(tournament_rounds):
    # Base URL for the tournament's hole stats (replace with your specific tournament ID and round ID)
    base_url = "https://www.pdga.com/api/v1/feat/stats/hole-stats-breakdown/"

    # Number of holes in the course
    num_holes = 18

    # List to store data for all holes
    all_holes_data = []
    for name, rids in tournament_rounds.items():
        for r, rid in rids.items():
            for hole_num in range(1, num_holes + 1):
                # Construct the full URL for the current hole
                url = f"{base_url}{rid}/{hole_num}"
                
                # Fetch the data
                print(f"Checking {name} round {r} hole {hole_num}")
                response = requests.get(url)
                
                if response.status_code == 404:
                    print(f"{name} round {r} hole {hole_num} not available")  # raises exception when not a 2xx response
                elif response.status_code != 204:
                    data = response.json()
                    # Extract and store stats for each player in the current hole
                    for entry in data:
                        player_info = entry['liveScore']['liveResult']
                        breakdown = entry['breakdown']
                        
                        # Combine player info with breakdown and add hole number
                        player_stats = {
                            'Tournament': name,
                            'Round': r,
                            'Hole': hole_num,
                            'FirstName': player_info['firstName'],
                            'LastName': player_info['lastName'],
                            'PDGANum': player_info['pdgaNum'],
                            'Place': player_info['place'],
                            'TotalScore': player_info['total'],
                            'ToPar': player_info['toPar'],
                            'Driving': breakdown.get('driving', None),
                            'Scramble': breakdown.get('scramble', None),
                            'Green': breakdown.get('green', None),
                            'C1x': breakdown.get('c1x', 0),
                            'C1': breakdown.get('c1', 0),
                            'C2': breakdown.get('c2', 0),
                            'ThrowIn': breakdown.get('throwIn', 0),
                            'OB': breakdown.get('ob', 0),
                            'Hazard': breakdown.get('hazard', 0),
                            'MissedMando': breakdown.get('missedMando', 0),
                            'LostDisc': breakdown.get('lostDisc', 0),
                            'Penalty': breakdown.get('penalty', 0),
                            'HoleScore': breakdown.get('holeScore', None),
                            'RID': rid,                    
                        }
                    all_holes_data.append(player_stats)
                else:
                    print(f"Data not available for {name} Round {r} hole {hole_num}")
    return all_holes_data

            

In [ ]:
check_dict = {'DGPT - All-Star Weekend - Doubles': {1: '122299849'}, 'DGPT - Chess.com Invitational presented by Discraft': {1: '122287100', 2: '122287102'}}
check_data = get_tournament_data(check_dict)

In [ ]:
check_df = pd.DataFrame(check_data)
print(check_df)

In [ ]:
def fetch_hole_data(base_url, rid, hole_num, tournament_name):
    """Fetch data for a specific hole."""
    url = f"{base_url}{rid}/{hole_num}"
    response = requests.get(url)
    
    if response.status_code == 404:
        print(f"{tournament_name} Round {rid} hole {hole_num} not available")
        return []
    elif response.status_code != 204:
        data = response.json()
        return data
    else:
        print(f"Data not available for Round {rid} hole {hole_num}")
        return []

def process_player_data(entry, tournament_name, round_number, hole_num, rid):
    """Process data for a single player."""
    player_info = entry['liveScore']['liveResult']
    breakdown = entry['breakdown']

    player_stats = {
        'Tournament': tournament_name,
        'Round': round_number,
        'Hole': hole_num,
        'FirstName': player_info['firstName'],
        'LastName': player_info['lastName'],
        'PDGANum': player_info['pdgaNum'],
        'Place': player_info['place'],
        'TotalScore': player_info['total'],
        'ToPar': player_info['toPar'],
        'Driving': breakdown.get('driving', None),
        'Scramble': breakdown.get('scramble', None),
        'Green': breakdown.get('green', None),
        'C1x': breakdown.get('c1x', 0),
        'C1': breakdown.get('c1', 0),
        'C2': breakdown.get('c2', 0),
        'ThrowIn': breakdown.get('throwIn', 0),
        'OB': breakdown.get('ob', 0),
        'Hazard': breakdown.get('hazard', 0),
        'MissedMando': breakdown.get('missedMando', 0),
        'LostDisc': breakdown.get('lostDisc', 0),
        'Penalty': breakdown.get('penalty', 0),
        'HoleScore': breakdown.get('holeScore', None),
        'RID': rid,                    
    }
    return player_stats

def get_tournament_data(tournament_rounds):
    """Main function to get data for all rounds in all tournaments."""
    base_url = "https://www.pdga.com/api/v1/feat/stats/hole-stats-breakdown/"
    num_holes = 18
    all_holes_data = []

    for tournament_name, rounds in tournament_rounds.items():
        for round_number, rid in rounds.items():
            for hole_num in range(1, num_holes + 1):
                hole_data = fetch_hole_data(base_url, rid, hole_num, tournament_name)
                for entry in hole_data:
                    player_stats = process_player_data(entry, tournament_name, round_number, hole_num, rid)
                    all_holes_data.append(player_stats)
    
    return all_holes_data

In [ ]:
tournament_data = get_tournament_data(check_dict)

In [ ]:
df = pd.DataFrame(tournament_data)
print(df)

In [ ]:
df.to_csv('parital_season_stats.csv', index=False)


In [ ]:
# Bypass if data already downloaded and formatted into csv. Execution took over 30 minutes
# tournament_data_full = get_tournament_data(cleaned_tournament_rounds)

In [ ]:
tournament_data_df = pd.DataFrame(tournament_data_full)

In [ ]:
tournament_data_df.to_csv('Full_Season_Stats.csv', index=False)

In [ ]:
print(tournament_data_df)

It looks like we are missing information about course layouts. Luckily, I was able to capture the api request that helps put together the results page on pdga.com, and we can extract information about the course layouts by parsing this data

In [ ]:
def get_course_info(tournament_id):
    url = f"https://www.pdga.com/apps/tournament/live-api/live_results_fetch_event?TournID={tournament_id}"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()['data']
        return data['Layouts']
    else:
        print(f"Failed to retrieve course info for Tournament ID {tournament_id}")
        return None

In [ ]:
def extract_hole_details(layouts):
    hole_info = {}
    
    for layout in layouts:
        layout_id = layout['LayoutID']
        course_name = layout['CourseName']
        
        for hole in layout['Details']:
            hole_number = hole['Label']
            par = hole['Par']
            length = hole['Length']
            
            hole_info[(layout_id, hole_number)] = {
                'CourseName': course_name,
                'Par': par,
                'Length': length
            }
    
    return hole_info


In [ ]:

def add_course_info_to_df(df, tournaments_dict):
    new_df = df.copy()
    new_df['CourseName'] = None
    new_df['Par'] = None
    new_df['Distance'] = None
    new_df['Hole'] = new_df['Hole'].astype(str)
    
    for tournament, tournament_id in tournaments_dict.items():
        # Fetch course info for this tournament
        url = f'https://www.pdga.com/apps/tournament/live-api/live_results_fetch_event?TournID={tournament_id}'
        response = requests.get(url)
        data = response.json()
        print(f"Data retrieved for {tournament}")
        
        # Extract LayoutAssignments and Layouts specifically for the MPO division
        layout_assignments = {}
        layouts = {}
        mpo_layouts = {}
        
        for division in data['data']['Divisions']:
            if division['Division'] == 'MPO':  # Filter only for MPO division
                mpo_layouts = division['LayoutAssignments']
                break  # Stop after finding MPO
        
        for layout in data['data']['Layouts']:
            layouts[layout['LayoutID']] = layout
        
        # Go through each round in the dataframe
        for round_num, layout_id in mpo_layouts.items():
            course_info = layouts[layout_id]
            course_name = course_info["CourseName"]
            for hole in course_info['Details']:
                hole_number = str(hole['Label'])
                
                # Account for possible non-numeric labels
                mask = (new_df['Tournament'] == tournament) & (new_df['Round'] == int(round_num))
                
                if hole_number.isnumeric():
                    mask &= (new_df['Hole'] == hole_number)
                else:
                    base_hole_number = hole_number[:-1] if len(hole_number) > 1 else hole_number
                    mask &= (new_df['Hole'].astype(str) == base_hole_number)
                    new_df.loc[mask, 'Hole'] = hole_number
                    print(f"Updating {tournament} round {round_num} hole {hole_number}")

                # Update the dataframe with course info
                new_df.loc[mask, 'Par'] = hole['Par']
                new_df.loc[mask, 'Distance'] = hole['Length']
                new_df.loc[mask, 'CourseName'] = course_name
                
    return new_df


In [ ]:
tournament_data_df_new = add_course_info_to_df(tournament_data_df, tournaments_dict)

In [ ]:
def calculate_scores(tournament_data_df):
    # Sort the dataframe to ensure proper order of tournaments, rounds, and holes
    df = tournament_data_df.copy()
    df = df.sort_values(by=['Tournament', 'PDGANum', 'Round'])

    # Initialize cumulative scores columns
    df['CumulativeRoundScore'] = 0
    df['CumulativeRoundScoreToPar'] = 0
    df['CumulativeTournamentScore'] = 0
    df['CumulativeTournamentScoreToPar'] = 0
    df['HoleScoreToPar'] = 0

    # Group by player and tournament to calculate scores for each player in each tournament separately
    for (tournament, pdga_num), player_df in df.groupby(['Tournament', 'PDGANum']):
        cumulative_round_score = 0
        cumulative_tournament_score = 0
        cumulative_round_score_to_par = 0
        cumulative_tournament_score_to_par = 0
        current_round = None

        for index, row in player_df.iterrows():
            # If the round changes, reset the cumulative round scores
            if current_round != row['Round']:
                current_round = row['Round']
                cumulative_round_score = 0
                cumulative_round_score_to_par = 0
        
            # Calculate the score for the current hole
            hole_score = row['HoleScore']
            hole_par = row['Par']
            if hole_par != None:
                score_to_par = hole_score - hole_par

                # Update cumulative scores
                cumulative_round_score += hole_score
                cumulative_round_score_to_par += score_to_par
                cumulative_tournament_score += hole_score
                cumulative_tournament_score_to_par += score_to_par

                # Assign calculated values to the dataframe
                df.at[index, 'CumulativeRoundScore'] = cumulative_round_score
                df.at[index, 'CumulativeRoundScoreToPar'] = cumulative_round_score_to_par
                df.at[index, 'CumulativeTournamentScore'] = cumulative_tournament_score
                df.at[index, 'CumulativeTournamentScoreToPar'] = cumulative_tournament_score_to_par
                df.at[index, 'HoleScoreToPar'] = score_to_par

    return df


In [ ]:
tournament_data_df_new = calculate_scores(tournament_data_df_new)

In [ ]:
tournament_data_df_new.to_csv('Full_Season_Stats.csv', index=False)

In [ ]:
# If tournament data is already exported to csv
tournament_data_df_new = pd.read_csv('Full_Season_Stats.csv')

In [ ]:

# Function to calculate summary stats by round

def summary_stats(round_df):
    # Initialize summary statistics dictionary
    summary_stats = {}

    # Group by player
    players = round_df.groupby(['FirstName', 'LastName', 'PDGANum'])

    # Process each player
    for player, data in players:
        c1x_makes = data[(data['ThrowIn'] >= 11) & (data['ThrowIn'] < 33)].shape[0]
        c1x_attempts = data['C1'].sum() - data[(data['ThrowIn'] < 11) & (data['ThrowIn'] != 0)].shape[0]
        
        c2_makes = data[(data['ThrowIn'] >= 33) & (data['ThrowIn'] <= 66)].shape[0]
        c2_attempts = data['C2'].sum()
        
        parked_holes = data[data['Green'] == 'parked'].shape[0]
        c1_driving_holes = data[(data['Green'] == 'parked') | (data['Green'] == 'c1')].shape[0]
        c2_driving_holes = data[(data['Green'] == 'parked') | (data['Green'] == 'c1') | (data['Green'] == 'c2')].shape[0]

        birdies = data[(data['HoleScoreToPar'] == -1)].shape[0]
        pars = data[(data['HoleScoreToPar'] == 0)].shape[0]
        bogeys = data[(data['HoleScoreToPar'] == 1)].shape[0]
        dbl_or_more = data[(data['HoleScoreToPar'] > 1)].shape[0]
        eagles = data[(data['HoleScoreToPar'] == -2)].shape[0]

        total_scramble_holes = data[(data['Scramble'] == 'success') | (data['Scramble'] == 'fail')].shape[0]
        successful_scrambles = data[(data['Scramble'] == 'success')].shape[0]
        total_holes = data.shape[0]
        
        round_score = data['HoleScore'].sum()
        round_score_to_par = data['HoleScoreToPar'].sum()

        average_throw_in_distance = data[data['ThrowIn'] > 0]['ThrowIn'].mean()
        
        summary_stats[player] = {
            'Score to Par': round_score_to_par,
            'Score': round_score,
            'C1x Putting %': (c1x_makes / c1x_attempts * 100) if c1x_attempts > 0 else None,
            'C2 Putting %': (c2_makes / c2_attempts * 100) if c2_attempts > 0 else None,
            'Parked %': (parked_holes / total_holes * 100) if total_holes > 0 else None,
            'C1 Driving %': (c1_driving_holes / total_holes * 100) if total_holes > 0 else None,
            'C2 Driving %': (c2_driving_holes / total_holes * 100) if total_holes > 0 else None,
            'Birdie %': (birdies / total_holes * 100) if total_holes > 0 else None,
            'Par %': (pars / total_holes * 100) if total_holes > 0 else None,
            'Bogey %': (bogeys / total_holes * 100) if total_holes > 0 else None,
            'Double Bogey or Worst %': (dbl_or_more / total_holes * 100) if total_holes > 0 else None,
            'Eagle %': (eagles / total_holes * 100) if total_holes > 0 else None,
            'Average Throw-In Distance': average_throw_in_distance,
            'C1x Attempts': c1x_attempts,
            'C1x Makes': c1x_makes,
            'C2 Attempts': c2_attempts,
            'C2 Makes': c2_makes,
            'C1 misses - C2 makes': (c1x_attempts-c1x_makes) - c2_makes,
            'Birdies': birdies,
            'Pars': pars,
            'Bogeys': bogeys,
            'Double Bogey or Worse': dbl_or_more,
            'Eagles': eagles,
            'Successful Scrambles': successful_scrambles,
            'Scramble Attempts': total_scramble_holes,
            'Scramble %': (successful_scrambles/total_scramble_holes) if total_scramble_holes > 0 else None,
            'Park Jobs': parked_holes,
            'C1 hits': c1_driving_holes,
            'C2 hits': c2_driving_holes,
        }

    # Convert to DataFrame
    summary_df = pd.DataFrame.from_dict(summary_stats, orient='index')
    summary_df = summary_df.reset_index()
    summary_df = summary_df.rename(columns={'level_0': 'FirstName', 'level_1': 'LastName', 'level_2': 'PDGANum'})

    return summary_df


In [ ]:
def calculate_all_rounds_stats(full_df):
    # Initialize list to store summary dataframes for all rounds
    all_summary_stats = []

    # Group by round
    rounds = full_df.groupby(['Tournament', 'Round'])

    # Process each round
    for (tournament, round_number), round_df in rounds:
        print(f"Processing Tournament: {tournament}, Round: {round_number}")
        round_summary_df = summary_stats(round_df)
        
        # Add Tournament and Round information to the summary dataframe
        round_summary_df['Tournament'] = tournament
        round_summary_df['Round'] = round_number
        
        all_summary_stats.append(round_summary_df)

    # Combine all summary dataframes into a single dataframe
    full_summary_df = pd.concat(all_summary_stats)

    return full_summary_df

In [ ]:
full_round_stats = calculate_all_rounds_stats(tournament_data_df_new)

In [ ]:
def aggregate_player_stats(summary_df):
    # Group by player across all rounds
    df = summary_df.copy()
    player_group = df.groupby(['FirstName', 'LastName', 'PDGANum'])

    # Initialize a list to store aggregated player stats
    player_stats_list = []

    # Process each player
    for player, data in player_group:
        total_birdies = data['Birdies'].sum()
        total_pars = data['Pars'].sum()
        total_bogeys = data['Bogeys'].sum()
        total_dbl_or_more = data['Double Bogey or Worse'].sum()
        total_eagles = data['Eagles'].sum()
        total_holes = total_birdies + total_pars + total_bogeys + total_dbl_or_more + total_eagles


        total_score = data['Score'].sum()
        total_score_to_par = data['Score to Par'].sum()
        avg_score_to_par = data['Score to Par'].mean()
        total_c1x_makes = data['C1x Makes'].sum()
        total_c1x_attempts = data['C1x Attempts'].sum()
        total_c2_makes = data['C2 Makes'].sum()
        total_c2_attempts = data['C2 Attempts'].sum()
        c1_miss_c2_make = (total_c1x_attempts-total_c1x_makes) - total_c2_makes
        total_parked_holes = data['Park Jobs'].sum()
        total_c1_driving_holes = data['C1 hits'].sum()
        total_c2_driving_holes = data['C2 hits'].sum()

        total_birdies = data['Birdies'].sum()
        total_pars = data['Pars'].sum()
        total_bogeys = data['Bogeys'].sum()
        total_dbl_or_more = data['Double Bogey or Worse'].sum()
        total_eagles = data['Eagles'].sum()
        total_holes = total_birdies + total_pars + total_bogeys + total_dbl_or_more + total_eagles

        total_successful_scrambles = data['Successful Scrambles'].sum()
        total_scramble_attempts = data['Scramble Attempts'].sum()

        # Calculate percentages based on totals
        c1x_putting_percentage = (total_c1x_makes / total_c1x_attempts * 100) if total_c1x_attempts > 0 else None
        c2_putting_percentage = (total_c2_makes / total_c2_attempts * 100) if total_c2_attempts > 0 else None
        scramble_percentage = (total_successful_scrambles / total_scramble_attempts * 100) if total_scramble_attempts > 0 else None

        # Append player stats to the list
        player_stats_list.append({
            'FirstName': player[0],
            'LastName': player[1],
            'PDGANum': player[2],
            'Total Score': total_score,
            'Rounds Played': round(total_score_to_par/avg_score_to_par, 0) if avg_score_to_par != 0 else None,
            'Total Score to Par': total_score_to_par,
            'Average Score to Par': round(avg_score_to_par, 2),
            'C1x Putting %': c1x_putting_percentage,
            'C2 Putting %': c2_putting_percentage,
            'C1 Misses - C2 Makes': c1_miss_c2_make,
            'Parked %': (total_parked_holes / total_holes * 100) if total_holes > 0 else None,
            'C1 Driving %': (total_c1_driving_holes / total_holes * 100) if total_holes > 0 else None,
            'C2 Driving %': (total_c2_driving_holes / total_holes * 100) if total_holes > 0 else None,
            'Birdie %': (total_birdies / total_holes * 100) if total_holes > 0 else None,
            'Par %': (total_pars / total_holes * 100) if total_holes > 0 else None,
            'Bogey %': (total_bogeys / total_holes * 100) if total_holes > 0 else None,
            'Double Bogey or Worse %': (total_dbl_or_more / total_holes * 100) if total_holes > 0 else None,
            'Eagle %': (total_eagles / total_holes * 100) if total_holes > 0 else None,
            'Scramble %': scramble_percentage
        })

    # Convert the list to a dataframe
    player_stats_df = pd.DataFrame(player_stats_list)

    # Rank players in each category
    for col in ['Total Score', 'Total Score to Par', 'C1x Putting %', 'C2 Putting %', 
                'Parked %', 'C1 Driving %', 'C2 Driving %', 
                'Birdie %', 'Par %', 'Bogey %', 
                'Double Bogey or Worse %', 'Eagle %', 'Scramble %', 'C1 Misses - C2 Makes']:
        player_stats_df[f'{col} Rank'] = player_stats_df[col].rank(method='min', ascending=False)

    return player_stats_df


In [ ]:
player_season_stats = aggregate_player_stats(full_round_stats)

In [ ]:
player_season_stats